In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.sparse import issparse, coo_matrix, csr_matrix, csc_matrix
from scipy.sparse.linalg import eigsh
from openfermionpyscf import run_pyscf
from openfermion import MolecularData, get_fermion_operator
from openfermion.linalg import get_sparse_operator, get_number_preserving_sparse_operator

In [2]:
def indices_with_N_and_Sz0(n_orbital, n_elec):
    """전자수=N 이면서 S_z=0 (interleaved: 짝수=alpha, 홀수=beta)"""
    idx = []
    for s in range(1 << n_orbital):
        if s.bit_count() != n_elec:
            continue
        n_alpha = sum((s >> i) & 1 for i in range(0, n_orbital, 2))  # 0,2,4,...
        n_beta  = sum((s >> i) & 1 for i in range(1, n_orbital, 2))  # 1,3,5,...
        if n_alpha == n_beta:
            idx.append(s)
    return np.array(idx, dtype=int)

def sparse_to_graph(A, *,
                    directed=False,
                    weight="value",   # "value" | "abs" | "binary"
                    symmetrize=True,  # 무향 그래프용: 패턴을 A + A.T로 합칠지
                    tol=0.0,          # |a_ij|<=tol 은 0 취급
                    keep_diagonal=False):
    """
    희소행렬 A -> NetworkX Graph/DiGraph 변환

    Parameters
    ----------
    A : scipy.sparse matrix (정방 추천)
    directed : bool
        True면 DiGraph, False면 Graph
    weight : {"value","abs","binary"}
        엣지 weight 설정 방법
        - "value": a_ij (실수/복소 가능; 복소는 실수부 사용 권장)
        - "abs"  : |a_ij|
        - "binary": 1 (연결만 표현)
    symmetrize : bool
        무향 그래프에서 A의 패턴을 A + A^T로 결합 (권장)
        directed=True 인 경우엔 무시됨
    tol : float
        임계값 이하 절댓값은 0으로 무시
    keep_diagonal : bool
        True면 i==j 항도 self-loop로 추가

    Returns
    -------
    G : nx.Graph or nx.DiGraph
    """
    if not issparse(A):
        raise ValueError("A must be a scipy.sparse matrix")
    # COO로 변환
    A = A.tocoo(copy=True)

    # tol 필터링
    if tol > 0:
        mask = np.abs(A.data) > tol
        A = coo_matrix((A.data[mask], (A.row[mask], A.col[mask])), shape=A.shape)

    # 대각 요소 처리
    if not keep_diagonal:
        mask = A.row != A.col
        A = coo_matrix((A.data[mask], (A.row[mask], A.col[mask])), shape=A.shape)

    # 무향이면 패턴 대칭화(권장): A <- A + A.T (중복은 합쳐짐)
    if not directed and symmetrize:
        AT = coo_matrix((A.data, (A.col, A.row)), shape=A.shape)
        A = (A + AT).tocoo()

    # 그래프 타입 선택
    G = nx.DiGraph() if directed else nx.Graph()
    G.add_nodes_from(range(A.shape[0]))  # 노드: 0..n-1

    # weight 설정
    if weight == "binary":
        # 동일 (i,j) 중복 합치기 위해 집계
        from collections import defaultdict
        edges = defaultdict(float)
        for i, j, v in zip(A.row, A.col, A.data):
            if not directed and i == j and not keep_diagonal:
                continue
            key = (i, j) if directed else (min(i, j), max(i, j))
            edges[key] = 1.0  # 존재만 표시
        for (i, j), w in edges.items():
            G.add_edge(i, j, weight=w)

    else:
        # "value" 또는 "abs"
        if weight == "abs":
            vals = np.abs(A.data)
        elif weight == "value":
            # 복소인 경우 실수부만 쓰고 싶다면 .real 사용
            # 필요에 따라 변경 가능: vals = np.real(A.data)
            vals = A.data
        else:
            raise ValueError("weight must be 'value', 'abs', or 'binary'")

        # 동일 엣지 중복 합치기(무향일 때 i<j 묶기)
        from collections import defaultdict
        edges = defaultdict(float)
        for i, j, v in zip(A.row, A.col, vals):
            if not directed and i == j and not keep_diagonal:
                continue
            key = (i, j) if directed else (min(i, j), max(i, j))
            edges[key] += float(v)  # 누적(합). 필요시 max/mean 등으로 변경 가능.

        for (i, j), w in edges.items():
            G.add_edge(i, j, weight=w)

    return G

In [3]:
def sparse_to_graph(A, *,
                    directed=False,
                    weight="value",   # "value" | "abs" | "binary"
                    symmetrize=True,  # 무향 그래프용: 패턴을 A + A.T로 합칠지
                    tol=0.0,          # |a_ij|<=tol 은 0 취급
                    keep_diagonal=False):
    """
    희소행렬 A -> NetworkX Graph/DiGraph 변환

    Parameters
    ----------
    A : scipy.sparse matrix (정방 추천)
    directed : bool
        True면 DiGraph, False면 Graph
    weight : {"value","abs","binary"}
        엣지 weight 설정 방법
        - "value": a_ij (실수/복소 가능; 복소는 실수부 사용 권장)
        - "abs"  : |a_ij|
        - "binary": 1 (연결만 표현)
    symmetrize : bool
        무향 그래프에서 A의 패턴을 A + A^T로 결합 (권장)
        directed=True 인 경우엔 무시됨
    tol : float
        임계값 이하 절댓값은 0으로 무시
    keep_diagonal : bool
        True면 i==j 항도 self-loop로 추가

    Returns
    -------
    G : nx.Graph or nx.DiGraph
    """
    if not issparse(A):
        raise ValueError("A must be a scipy.sparse matrix")
    # COO로 변환
    A = A.tocoo(copy=True)

    # tol 필터링
    if tol > 0:
        mask = np.abs(A.data) > tol
        A = coo_matrix((A.data[mask], (A.row[mask], A.col[mask])), shape=A.shape)

    # 대각 요소 처리
    if not keep_diagonal:
        mask = A.row != A.col
        A = coo_matrix((A.data[mask], (A.row[mask], A.col[mask])), shape=A.shape)

    # 무향이면 패턴 대칭화(권장): A <- A + A.T (중복은 합쳐짐)
    if not directed and symmetrize:
        AT = coo_matrix((A.data, (A.col, A.row)), shape=A.shape)
        A = (A + AT).tocoo()

    # 그래프 타입 선택
    G = nx.DiGraph() if directed else nx.Graph()
    G.add_nodes_from(range(A.shape[0]))  # 노드: 0..n-1

    # weight 설정
    if weight == "binary":
        # 동일 (i,j) 중복 합치기 위해 집계
        from collections import defaultdict
        edges = defaultdict(float)
        for i, j, v in zip(A.row, A.col, A.data):
            if not directed and i == j and not keep_diagonal:
                continue
            key = (i, j) if directed else (min(i, j), max(i, j))
            edges[key] = 1.0  # 존재만 표시
        for (i, j), w in edges.items():
            G.add_edge(i, j, weight=w)

    else:
        # "value" 또는 "abs"
        if weight == "abs":
            vals = np.abs(A.data)
        elif weight == "value":
            # 복소인 경우 실수부만 쓰고 싶다면 .real 사용
            # 필요에 따라 변경 가능: vals = np.real(A.data)
            vals = A.data
        else:
            raise ValueError("weight must be 'value', 'abs', or 'binary'")

        # 동일 엣지 중복 합치기(무향일 때 i<j 묶기)
        from collections import defaultdict
        edges = defaultdict(float)
        for i, j, v in zip(A.row, A.col, vals):
            if not directed and i == j and not keep_diagonal:
                continue
            key = (i, j) if directed else (min(i, j), max(i, j))
            edges[key] += float(v)  # 누적(합). 필요시 max/mean 등으로 변경 가능.

        for (i, j), w in edges.items():
            G.add_edge(i, j, weight=w)

    return G

In [4]:
# #H12
# H12_energy_arr = []
# for dist in np.arange(0.4,2.5, 0.1):


In [5]:
dist = 0.9
geometry = [('H', (0., 0., 0.)),
            ('H', (dist, 0., 0. )),
            ('H', (2*dist, 0., 0. )),
            ('H', (3*dist, 0., 0.)),
            ('H', (4*dist, 0., 0.)),
            ('H', (5*dist, 0., 0. )),
            ('H', (6*dist, 0., 0. )),
            ('H', (7*dist, 0., 0.)),
            ('H', (8*dist, 0., 0.)),
            ('H', (9*dist, 0., 0. )),
            ('H', (10*dist, 0., 0. )),
            ('H', (11*dist, 0., 0.))]
basis = 'sto-3g'
mol = MolecularData(geometry, basis, multiplicity=1, charge=0)
mol = run_pyscf(mol, run_scf=1, run_fci=0)
ham_int = mol.get_molecular_hamiltonian()
ham_fci = get_fermion_operator(ham_int)  
print(ham_fci)

14.83961547563585 [] +
-3.2897247162353755 [0^ 0] +
0.1640673456647648 [0^ 0^ 0 0] +
-0.032998526327466006 [0^ 0^ 0 4] +
-0.0007421627814093455 [0^ 0^ 0 8] +
-0.00011855424196707754 [0^ 0^ 0 12] +
-0.00020130306141312058 [0^ 0^ 0 16] +
-0.00045094229074876065 [0^ 0^ 0 20] +
0.0594117270704862 [0^ 0^ 2 2] +
0.0234913333973451 [0^ 0^ 2 6] +
-0.0013937348857660718 [0^ 0^ 2 10] +
0.000635823849233111 [0^ 0^ 2 14] +
-0.0005979608466465686 [0^ 0^ 2 18] +
-0.0002594164819528269 [0^ 0^ 2 22] +
-0.03299852632746602 [0^ 0^ 4 0] +
0.04298130553305152 [0^ 0^ 4 4] +
0.018508269479569257 [0^ 0^ 4 8] +
-0.0019406500523477066 [0^ 0^ 4 12] +
0.0007741876093255851 [0^ 0^ 4 16] +
0.0002782957183178394 [0^ 0^ 4 20] +
0.023491333397345104 [0^ 0^ 6 2] +
0.03286760348025724 [0^ 0^ 6 6] +
-0.01565174945086317 [0^ 0^ 6 10] +
0.000884451332523638 [0^ 0^ 6 14] +
-0.0002977648325017937 [0^ 0^ 6 18] +
7.361439719944717e-06 [0^ 0^ 6 22] +
-0.0007421627814093516 [0^ 0^ 8 0] +
0.018508269479569246 [0^ 0^ 8 4] +
0.025

In [12]:
H_N_Sz = get_number_preserving_sparse_operator(
    fermion_op=ham_fci,
    num_qubits=mol.n_qubits,
    num_electrons=mol.n_electrons,       # 필수
    spin_preserving=True ,        # S_z 고정 (필요 없으면 None)
    reference_determinant=None,
    excitation_level=None
)

KeyboardInterrupt: 

In [11]:
H = get_sparse_operator(ham_fci, n_qubits=mol.n_qubits)

: 

In [ ]:

# 3. 2차 정량화 Hamiltonian 얻기
   

print(H)

In [ ]:
def GBBD(mol,n_e, istwo = False):
    mol = run_pyscf(mol, run_scf=1, run_fci=0)
    # 3. 2차 정량화 Hamiltonian 얻기
    ham_int = mol.get_molecular_hamiltonian()
    ham_fci = get_fermion_operator(ham_int)     
    H = get_sparse_operator(ham_fci, n_qubits=mol.n_qubits)
    print(H)
    if istwo == True : 
        eigval, eigvec = eigsh(H, which='SA') 
    else : 
        # 2) N=2, S_z=0 서브스페이스 (H2/STO-3G면 4x4)
        idx_N2_Sz0 = indices_with_N_and_Sz0(mol.n_qubits, n_elec=n_e)
        #print(idx_N2_Sz0)
        H_cons = H[idx_N2_Sz0, :][:, idx_N2_Sz0].tocsr()
        H_real = H_cons.real
        #print(H_cons)
        G = sparse_to_graph(H_real, directed=False, weight="abs", symmetrize=True, tol=0.0)
        # 연결 성분
        ccs = list(nx.connected_components(G))
        sub_mat_idx = list(ccs[0])
        H_sub = H_cons[sub_mat_idx, :][:, sub_mat_idx]
        print(H_sub)
        eigval, eigvec = eigsh(H_sub, k=1, which='SA') 
    return eigval[0]
   